In [1]:
import os
import numpy as np
import itertools
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 20)

<br>

# Functions

In [2]:
def M2Q(df_in):
    d = {"M03":"Q1", "M06":"Q2", "M09":"Q3", "M12":"Q4"}
    df_out = df_in[df_in['TIME'].apply(lambda x: x[-3:] in list(d.keys()))].copy()
    df_out.TIME = df_out['TIME'].apply(lambda x: x[:4]+d[x[-3:]]).copy()
    return(df_out)

def Qp1(df_in):
    d = {"Q1":"Q2", "Q2":"Q3", "Q3":"Q4", "Q4":"Q1"}
    df_out = df_in.copy()
    df_out.TIME = df_out['TIME'].apply(lambda x: str(int(x[:-2])+1)+d[x[-2:]] if x[-1:] == "4" else x[:-2]+d[x[-2:]]).copy()
    return(df_out)

<br>

# Data

In [10]:
!sed -i "s/://g" ./data/*

sed: 1: "./data/ei_bsci_m_r2_1_D ...": invalid command code .


In [6]:
#!head -n 1 data/sts_trtu_q_Label.csv

>## Quarterly

In [7]:
# DATASET Construction - quarterly data - index (2015 = 100) (NACE Rev. 2) [ei_isbu_q]
ei_isbu_q = pd.read_csv("./data/ei_isbu_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
ei_isbu_q_lag = Qp1(ei_isbu_q.copy())

# DATASET Final consumption aggregates [namq_10_fcs]
namq_10_fcs = pd.read_csv("./data/namq_10_fcs_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
namq_10_fcs_lag = Qp1(namq_10_fcs.copy())

# DATASET: GDP and main components  (output, expenditure and income) [namq_10_gdp]
namq_10_gdp = pd.read_csv("./data/namq_10_gdp_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
namq_10_gdp_lag = Qp1(namq_10_gdp.copy())

# DATASET Building permits - quarterly data [sts_cobp_q]
sts_cobp_q = pd.read_csv("./data/sts_cobp_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_cobp_q_lag = Qp1(sts_cobp_q.copy())

# DATASET Production in construction - quarterly data [sts_copr_q]
sts_copr_q = pd.read_csv("./data/sts_copr_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_copr_q_lag = Qp1(sts_copr_q.copy())

# DATASET Production in industry - quarterly data [sts_inpr_q]
sts_inpr_q = pd.read_csv("./data/sts_inpr_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_inpr_q_lag = Qp1(sts_inpr_q.copy())

# DATASET Turnover in industry, total - quarterly data [sts_intv_q]
sts_intv_q = pd.read_csv("./data/sts_intv_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_intv_q_lag = Qp1(sts_intv_q.copy())

# DATASET Turnover and volume of sales in wholesale and retail trade - quarterly data [sts_trtu_q]
sts_trtu_q = pd.read_csv("./data/sts_trtu_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_trtu_q_lag = Qp1(sts_trtu_q.copy())

>## Monthly

In [8]:
# DATASET Euro-zone Business Climate Indicator - monthly data [ei_bsci_m_r2]
ei_bsci_m_r2_1 = M2Q(pd.read_csv("./data/ei_bsci_m_r2_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna())

# DATASET Sentiment indicators - monthly data [ei_bssi_m_r2]
ei_bssi_m_r2 = M2Q(pd.read_csv("./data/ei_bssi_m_r2_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna())

# DATASET Effective exchange rates indices - monthly data [ei_mfef_m]
ei_mfef_m = M2Q(pd.read_csv("./data/ei_mfef_m_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna())

**Dealing with ids:**

In [9]:
for df in [ei_isbu_q, ei_isbu_q_lag, namq_10_fcs, namq_10_fcs_lag, namq_10_gdp, 
           namq_10_gdp_lag, sts_cobp_q, sts_cobp_q_lag, sts_copr_q, sts_copr_q_lag, 
           sts_inpr_q, sts_inpr_q_lag, sts_intv_q, sts_intv_q_lag, sts_trtu_q, 
           sts_trtu_q_lag, ei_isbu_q, ei_isbu_q_lag, namq_10_fcs, namq_10_fcs_lag, 
           namq_10_gdp, namq_10_gdp_lag, sts_cobp_q, sts_cobp_q_lag, sts_copr_q, sts_copr_q_lag, 
           sts_inpr_q, sts_inpr_q_lag, sts_intv_q, sts_intv_q_lag, sts_trtu_q, sts_trtu_q_lag, ei_bsci_m_r2_1, ei_bssi_m_r2, ei_mfef_m]:
    df["ID"] = df["TIME"]+df["GEO"]
    df.Value = df.Value.replace(regex=r'[,]+', value='').astype(np.float)
    


ValueError: could not convert string to float: ':'

># OECD

In [8]:
def OECD():
    GEO_LABEL2GEO = dict(zip(namq_10_gdp.GEO_LABEL.values, namq_10_gdp.GEO.values)); GEO_LABEL2GEO["Germany"] = "DE"
    df_in = pd.read_csv("data/KEI_13112018143436794_[v1-oecd].csv")
    df_in["GEO"] = df_in.Country.map(GEO_LABEL2GEO)
    df_in = df_in[df_in.GEO.notnull()]
    df_in["ID"] = df_in.TIME.str[:4]+df_in.TIME.str[-2:]+df_in.GEO
    return(df_in)

In [9]:
OECD = OECD()

<br>

# $C$

>## $Cd$

In [10]:
def Cd_Cheat(n):
    Cnd_vars = ["BS-BCI", "BS-RCI-BAL", "BS-SCI-BAL", "BS-CSMCI-BAL", 
                "MIG_DCOG", "C29_C30", "LRHUTTTT", "SLRTCR03"] 
    #Cnd_vars = ["BS-RCI-BAL"," BS-SCI-BAL", "C29_C30", "SLRTCR03", "BS-CSMCI-BAL"]
    fake_ns = list(itertools.combinations(Cnd_vars, n))
    fake_df = pd.DataFrame(index=["variables", "R2", "R2Adj"])
    counter = 0
    
    for fake_n in fake_ns:


        df_in = pd.DataFrame(sorted(list(set(namq_10_fcs.ID))), columns=["ID"])
        
        ###########################################################################################
        
        # Chain linked volumes, percentage change compared to same period in previous year
        var = "CLV_PCH_SM"
        selec = namq_10_fcs[(namq_10_fcs.NA_ITEM=="P311_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
        df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Chain linked volumes, percentage change on previous period
        var = "CLV_PCH_PRE"
        selec = namq_10_fcs_lag[(namq_10_fcs_lag.NA_ITEM=="P311_S14")&(namq_10_fcs_lag.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
        df_in = pd.merge(df_in, selec, on='ID', how="inner")

        # Business Climate Indicator
        var = "BS-BCI"
        if var in fake_n:
            dic_BCI = dict(zip(ei_bsci_m_r2_1.TIME.values, ei_bsci_m_r2_1.Value.values))
            df_in[var] = df_in.ID.str[:-2].map(dic_BCI)
        
        # Retail Confidence Indicator
        var = "BS-RCI-BAL"
        if var in fake_n:
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Services Confidence Indicator
        var = "BS-SCI-BAL"
        if var in fake_n:
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Consumer Confidence Indicator
        var = "BS-CSMCI-BAL"
        if var in fake_n:
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Production in industry: MIG - Durable Consumer Goods
        var = "MIG_DCOG"
        if var in fake_n:
            selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Manufacture of motor vehicles, trailers, semi-trailers and of other transport equipment
        var = "C29_C30"
        if var in fake_n:
            selec = sts_intv_q[(sts_intv_q.NACE_R2==var)&(sts_intv_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Harmonised unemployment rate all persons, s.a.
        var = "LRHUTTTT"
        if var in fake_n:
            selec = OECD[OECD.SUBJECT==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")

        # Passenger car registrations, s.a.
        var = "SLRTCR03"
        if var in fake_n:
            selec = OECD[OECD.SUBJECT==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        ###########################################################################################
        
        # Dummies
        countries = set(namq_10_fcs.GEO); countries.remove('PT')
        for country in sorted(list(countries)):
            df_in[country] = (df_in.ID.str[-2:] == country).astype(int)
        
        Cd = df_in.dropna().copy(); Cd.set_index("ID", inplace=True)
        Cd = Cd.loc[:, (Cd != 0).any(axis=0)].copy()
        del var, selec, df_in
        
        ###########################################################################################
        
        # Save
        Cd.to_csv("./Cd.csv")
        
        ###########################################################################################
        
        # Regression
        X = sm.add_constant(Cd.loc[:, Cd.columns != "CLV_PCH_SM"].copy())
        y = Cd.loc[:, "CLV_PCH_SM"].copy()
        results = sm.OLS(y, X).fit()
        
        fake_df[counter] = [fake_n, results.rsquared, results.rsquared_adj]
        counter += 1
        if counter % 100 == 0:
            print(counter/len(fake_ns)*100)
        
    return(fake_df.transpose())

In [11]:
Cd_Cheat(1).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables        R2     R2Adj
2    (BS-SCI-BAL,)  0.481004  0.473805
3  (BS-CSMCI-BAL,)  0.418163  0.410994
5       (C29_C30,)  0.416586  0.407958
0        (BS-BCI,)  0.396543  0.388335
7      (SLRTCR03,)  0.385289  0.377875
1    (BS-RCI-BAL,)  0.383209  0.375543
4      (MIG_DCOG,)  0.356479  0.348044
6      (LRHUTTTT,)  0.326081  0.318323

In [12]:
Cd_Cheat(2).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables        R2     R2Adj
10       (BS-RCI-BAL, C29_C30)  0.537304  0.529921
15       (BS-SCI-BAL, C29_C30)  0.517902  0.509744
17      (BS-SCI-BAL, SLRTCR03)  0.516009  0.509055
7     (BS-RCI-BAL, BS-SCI-BAL)  0.514052  0.506914
19     (BS-CSMCI-BAL, C29_C30)   0.51259  0.504842
16      (BS-SCI-BAL, LRHUTTTT)  0.502469  0.495362
1         (BS-BCI, BS-SCI-BAL)  0.498789    0.4908
13  (BS-SCI-BAL, BS-CSMCI-BAL)  0.496719  0.489416
12      (BS-RCI-BAL, SLRTCR03)   0.48666  0.479511
14      (BS-SCI-BAL, MIG_DCOG)  0.486335    0.4785
..                         ...       ...       ...
6           (BS-BCI, SLRTCR03)  0.449817  0.441637
18    (BS-CSMCI-BAL, MIG_DCOG)  0.446197  0.438356
20    (BS-CSMCI-BAL, LRHUTTTT)  0.444205  0.437037
25         (C29_C30, LRHUTTTT)  0.422023  0.412935
22         (MIG_DCOG, C29_C30)  0.419677   0.41059
3           (BS-BCI, MIG_DCOG)  0.418974  0.410338
5           (BS-BCI, LRHUTTTT)   0.41629  0.407967
27        (LRHUTTTT, SLRTCR03)  0.392385  0.384677
24        (MIG_DCOG, SLRTCR03)  0.393128  0.384548
23        (MIG_DCOG, LRHUTTTT)  0.362001  0.353299

[28 rows x 3 columns]

In [13]:
Cd_Cheat(3).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables        R2     R2Adj
34      (BS-RCI-BAL, C29_C30, SLRTCR03)   0.58968  0.582286
33      (BS-RCI-BAL, C29_C30, LRHUTTTT)  0.559068  0.551518
44      (BS-SCI-BAL, C29_C30, SLRTCR03)  0.557636  0.549586
23    (BS-RCI-BAL, BS-SCI-BAL, C29_C30)   0.55649  0.548609
27  (BS-RCI-BAL, BS-CSMCI-BAL, C29_C30)  0.554566  0.546928
50    (BS-CSMCI-BAL, C29_C30, SLRTCR03)  0.553004  0.545246
25   (BS-RCI-BAL, BS-SCI-BAL, SLRTCR03)  0.547215  0.540364
3         (BS-BCI, BS-RCI-BAL, C29_C30)  0.547991  0.540174
37  (BS-SCI-BAL, BS-CSMCI-BAL, C29_C30)  0.540503   0.53235
24   (BS-RCI-BAL, BS-SCI-BAL, LRHUTTTT)  0.538939  0.532003
..                                  ...       ...       ...
15          (BS-BCI, MIG_DCOG, C29_C30)  0.478595  0.469725
55        (C29_C30, LRHUTTTT, SLRTCR03)   0.47793  0.468892
53        (MIG_DCOG, C29_C30, SLRTCR03)  0.475757  0.466575
47   (BS-CSMCI-BAL, MIG_DCOG, LRHUTTTT)  0.472232   0.46446
17         (BS-BCI, MIG_DCOG, SLRTCR03)   0.46483  0.456106
11     (BS-BCI, BS-CSMCI-BAL, MIG_DCOG)   0.46413  0.455592
20         (BS-BCI, LRHUTTTT, SLRTCR03)  0.453169  0.444674
16         (BS-BCI, MIG_DCOG, LRHUTTTT)  0.437859  0.429034
52        (MIG_DCOG, C29_C30, LRHUTTTT)  0.425696  0.416102
54       (MIG_DCOG, LRHUTTTT, SLRTCR03)  0.399415  0.390526

[56 rows x 3 columns]

In [14]:
Cd_Cheat(4).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables        R2     R2Adj
43     (BS-RCI-BAL, BS-SCI-BAL, C29_C30, SLRTCR03)  0.604487  0.596909
49   (BS-RCI-BAL, BS-CSMCI-BAL, C29_C30, SLRTCR03)  0.599914  0.592337
13         (BS-BCI, BS-RCI-BAL, C29_C30, SLRTCR03)  0.599118  0.591251
54       (BS-RCI-BAL, C29_C30, LRHUTTTT, SLRTCR03)  0.596705  0.589068
52       (BS-RCI-BAL, MIG_DCOG, C29_C30, SLRTCR03)  0.590744  0.582994
42     (BS-RCI-BAL, BS-SCI-BAL, C29_C30, LRHUTTTT)  0.582895  0.574997
59   (BS-SCI-BAL, BS-CSMCI-BAL, C29_C30, SLRTCR03)  0.576659  0.568563
48   (BS-RCI-BAL, BS-CSMCI-BAL, C29_C30, LRHUTTTT)  0.574757  0.566958
12         (BS-BCI, BS-RCI-BAL, C29_C30, LRHUTTTT)   0.57072  0.562707
62       (BS-SCI-BAL, MIG_DCOG, C29_C30, SLRTCR03)  0.565959  0.557657
..                                             ...       ...       ...
27      (BS-BCI, BS-CSMCI-BAL, MIG_DCOG, SLRTCR03)    0.5053  0.496764
30      (BS-BCI, BS-CSMCI-BAL, LRHUTTTT, SLRTCR03)  0.502992  0.494794
46  (BS-RCI-BAL, BS-CSMCI-BAL, MIG_DCOG, LRHUTTTT)  0.501851   0.49395
53      (BS-RCI-BAL, MIG_DCOG, LRHUTTTT, SLRTCR03)  0.501447  0.493328
5     (BS-BCI, BS-RCI-BAL, BS-CSMCI-BAL, MIG_DCOG)  0.497063  0.488392
26      (BS-BCI, BS-CSMCI-BAL, MIG_DCOG, LRHUTTTT)  0.490997   0.48244
67    (BS-CSMCI-BAL, MIG_DCOG, LRHUTTTT, SLRTCR03)  0.489469   0.48141
31           (BS-BCI, MIG_DCOG, C29_C30, LRHUTTTT)  0.489879  0.480565
69         (MIG_DCOG, C29_C30, LRHUTTTT, SLRTCR03)  0.481492   0.47197
33          (BS-BCI, MIG_DCOG, LRHUTTTT, SLRTCR03)  0.466807  0.457725

[70 rows x 3 columns]

>## $Cnd$

In [15]:
def Cnd_Cheat(n):
    Cnd_vars = ["BS-BCI", "BS-RCI-BAL", "BS-SCI-BAL", "BS-CSMCI-BAL", "MIG_NDCOG", "G47_FOOD", "G47_NFOOD_X_G473", "G47_X_G473", "LRHUTTTT"]
    #Cnd_vars = ["BS-RCI-BAL", "BS-SCI-BAL", "G47_NFOOD_X_G473", "LRHUTTTT", "G47_FOOD"]
    fake_ns = list(itertools.combinations(Cnd_vars, n))
    fake_df = pd.DataFrame(index=["variables", "R2", "R2Adj"])
    counter = 0
    
    for fake_n in fake_ns:
        df_in = pd.DataFrame(sorted(list(set(namq_10_fcs.ID))), columns=["ID"])
        
        ###########################################################################################
    
        # Chain linked volumes, percentage change compared to same period in previous year
        var = "CLV_PCH_SM"
        selec = namq_10_fcs[(namq_10_fcs.NA_ITEM=="P312N_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
        df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Chain linked volumes, percentage change on previous period
        var = "CLV_PCH_PRE"
        selec = namq_10_fcs_lag[(namq_10_fcs_lag.NA_ITEM=="P312N_S14")&(namq_10_fcs_lag.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
        df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Business Climate Indicator
        var = "BS-BCI"
        if var in fake_n:    
            dic_BCI = dict(zip(ei_bsci_m_r2_1.TIME.values, ei_bsci_m_r2_1.Value.values))
            df_in[var] = df_in.ID.str[:-2].map(dic_BCI)
        
        # Retail Confidence Indicator
        var = "BS-RCI-BAL"
        if var in fake_n:    
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Services Confidence Indicator
        var = "BS-SCI-BAL"
        if var in fake_n:    
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Consumer Confidence Indicator
        var = "BS-CSMCI-BAL"
        if var in fake_n:    
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Production in industry: MIG - Non-Durable Consumer Goods
        var = "MIG_NDCOG"
        if var in fake_n:    
            selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Retail sale of food, beverages and tobacco
        var = "G47_FOOD"
        if var in fake_n:    
            selec = sts_trtu_q[sts_trtu_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Retail sale of non-food products (except fuel)
        var = "G47_NFOOD_X_G473"
        if var in fake_n:    
            selec = sts_trtu_q[sts_trtu_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Retail trade, except of motor vehicles, motorcyles and fuel
        var = "G47_X_G473"
        if var in fake_n:    
            selec = sts_trtu_q[sts_trtu_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Harmonised unemployment rate all persons, s.a.
        var = "LRHUTTTT"
        if var in fake_n:    
            selec = OECD[OECD.SUBJECT==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        ###########################################################################################
        
        # Dummies
        countries = set(namq_10_fcs.GEO); countries.remove('PT')
        for country in sorted(list(countries)):
            df_in[country] = (df_in.ID.str[-2:] == country).astype(int)
        
        Cnd = df_in.dropna().copy().set_index("ID")
        Cnd = Cnd.loc[:, (Cnd != 0).any(axis=0)].copy()
        del var, selec, df_in
        
        ###########################################################################################
        
        # Save
        #Cnd.to_csv("./Cnd.csv")
        
        ###########################################################################################
        
        # Regression
        X = sm.add_constant(Cnd.loc[:, Cnd.columns != "CLV_PCH_SM"].copy())
        y = Cnd.loc[:, "CLV_PCH_SM"].copy()
        results = sm.OLS(y, X).fit()
        
        fake_df[counter] = [fake_n, results.rsquared, results.rsquared_adj]
        counter += 1
        if counter % 100 == 0:
            print(counter/len(fake_ns)*100)
        
    return(fake_df.transpose())

In [16]:
Cnd_Cheat(1).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables        R2     R2Adj
2        (BS-SCI-BAL,)  0.646518  0.641615
3      (BS-CSMCI-BAL,)  0.563141  0.557759
8          (LRHUTTTT,)  0.493442  0.487611
1        (BS-RCI-BAL,)  0.466084  0.459448
0            (BS-BCI,)   0.46114   0.45381
6  (G47_NFOOD_X_G473,)  0.457069  0.453402
7        (G47_X_G473,)  0.457059  0.453392
5          (G47_FOOD,)  0.457058   0.45339
4         (MIG_NDCOG,)   0.44953  0.442302

In [17]:
Cnd_Cheat(2).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables        R2     R2Adj
8         (BS-RCI-BAL, BS-SCI-BAL)  0.674448  0.669667
15      (BS-SCI-BAL, BS-CSMCI-BAL)   0.66505  0.660189
20          (BS-SCI-BAL, LRHUTTTT)  0.662248  0.657423
1             (BS-BCI, BS-SCI-BAL)  0.656395  0.650918
18  (BS-SCI-BAL, G47_NFOOD_X_G473)  0.650834  0.648094
19        (BS-SCI-BAL, G47_X_G473)  0.650831   0.64809
17          (BS-SCI-BAL, G47_FOOD)   0.65083  0.648089
16         (BS-SCI-BAL, MIG_NDCOG)  0.651009  0.645686
14          (BS-RCI-BAL, LRHUTTTT)  0.629911  0.625025
9       (BS-RCI-BAL, BS-CSMCI-BAL)  0.620552   0.61536
..                             ...       ...       ...
6             (BS-BCI, G47_X_G473)  0.512213  0.508636
5       (BS-BCI, G47_NFOOD_X_G473)  0.512212  0.508634
4               (BS-BCI, G47_FOOD)   0.51221  0.508632
3              (BS-BCI, MIG_NDCOG)   0.50618  0.498864
33  (G47_NFOOD_X_G473, G47_X_G473)  0.462399  0.460507
30    (G47_FOOD, G47_NFOOD_X_G473)  0.462399  0.460507
31          (G47_FOOD, G47_X_G473)  0.462389  0.460497
28         (MIG_NDCOG, G47_X_G473)  0.464299  0.460379
26           (MIG_NDCOG, G47_FOOD)  0.464298  0.460379
27   (MIG_NDCOG, G47_NFOOD_X_G473)  0.464296  0.460377

[36 rows x 3 columns]

In [18]:
Cnd_Cheat(3).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables        R2     R2Adj
31    (BS-RCI-BAL, BS-SCI-BAL, G47_NFOOD_X_G473)  0.682445  0.679808
32          (BS-RCI-BAL, BS-SCI-BAL, G47_X_G473)  0.682442  0.679806
30            (BS-RCI-BAL, BS-SCI-BAL, G47_FOOD)  0.682438  0.679802
33            (BS-RCI-BAL, BS-SCI-BAL, LRHUTTTT)  0.683151  0.678385
28        (BS-RCI-BAL, BS-SCI-BAL, BS-CSMCI-BAL)  0.682032  0.677155
0               (BS-BCI, BS-RCI-BAL, BS-SCI-BAL)  0.682089  0.676739
29           (BS-RCI-BAL, BS-SCI-BAL, MIG_NDCOG)  0.677048  0.671843
7             (BS-BCI, BS-SCI-BAL, BS-CSMCI-BAL)  0.672845    0.6674
12                (BS-BCI, BS-SCI-BAL, LRHUTTTT)   0.67256  0.667141
51  (BS-SCI-BAL, BS-CSMCI-BAL, G47_NFOOD_X_G473)  0.669617   0.66691
..                                           ...       ...       ...
23                (BS-BCI, G47_FOOD, G47_X_G473)  0.516981  0.515139
25        (BS-BCI, G47_NFOOD_X_G473, G47_X_G473)  0.516979  0.515138
22          (BS-BCI, G47_FOOD, G47_NFOOD_X_G473)  0.516978  0.515137
20               (BS-BCI, MIG_NDCOG, G47_X_G473)  0.517887  0.514116
19         (BS-BCI, MIG_NDCOG, G47_NFOOD_X_G473)  0.517885  0.514115
18                 (BS-BCI, MIG_NDCOG, G47_FOOD)  0.517883  0.514113
75             (MIG_NDCOG, G47_FOOD, G47_X_G473)  0.470058  0.468042
74       (MIG_NDCOG, G47_FOOD, G47_NFOOD_X_G473)  0.470054  0.468038
77     (MIG_NDCOG, G47_NFOOD_X_G473, G47_X_G473)  0.470053  0.468037
80      (G47_FOOD, G47_NFOOD_X_G473, G47_X_G473)  0.465128   0.46415

[84 rows x 3 columns]

In [19]:
Cnd_Cheat(4).sort_values(["R2Adj", "R2"], ascending=[False, False])

79.36507936507937


variables        R2  \
69       (BS-RCI-BAL, BS-SCI-BAL, G47_NFOOD_X_G473, LRHUTTTT)  0.692225   
70             (BS-RCI-BAL, BS-SCI-BAL, G47_X_G473, LRHUTTTT)   0.69214   
67               (BS-RCI-BAL, BS-SCI-BAL, G47_FOOD, LRHUTTTT)   0.69207   
58   (BS-RCI-BAL, BS-SCI-BAL, BS-CSMCI-BAL, G47_NFOOD_X_G473)  0.689334   
59         (BS-RCI-BAL, BS-SCI-BAL, BS-CSMCI-BAL, G47_X_G473)  0.689316   
57           (BS-RCI-BAL, BS-SCI-BAL, BS-CSMCI-BAL, G47_FOOD)  0.689299   
5                  (BS-BCI, BS-RCI-BAL, BS-SCI-BAL, LRHUTTTT)  0.691804   
3          (BS-BCI, BS-RCI-BAL, BS-SCI-BAL, G47_NFOOD_X_G473)  0.687787   
4                (BS-BCI, BS-RCI-BAL, BS-SCI-BAL, G47_X_G473)  0.687785   
2                  (BS-BCI, BS-RCI-BAL, BS-SCI-BAL, G47_FOOD)  0.687781   
..                                                        ...       ...   
117    (BS-CSMCI-BAL, G47_FOOD, G47_NFOOD_X_G473, G47_X_G473)  0.571661   
124       (MIG_NDCOG, G47_NFOOD_X_G473, G47_X_G473, LRHUTTTT)  0.541926   
122         (MIG_NDCOG, G47_FOOD, G47_NFOOD_X_G473, LRHUTTTT)  0.541885   
123               (MIG_NDCOG, G47_FOOD, G47_X_G473, LRHUTTTT)  0.541852   
125        (G47_FOOD, G47_NFOOD_X_G473, G47_X_G473, LRHUTTTT)  0.536226   
47                  (BS-BCI, MIG_NDCOG, G47_FOOD, G47_X_G473)  0.522545   
49          (BS-BCI, MIG_NDCOG, G47_NFOOD_X_G473, G47_X_G473)  0.522544   
46            (BS-BCI, MIG_NDCOG, G47_FOOD, G47_NFOOD_X_G473)  0.522543   
52           (BS-BCI, G47_FOOD, G47_NFOOD_X_G473, G47_X_G473)  0.519427   
121       (MIG_NDCOG, G47_FOOD, G47_NFOOD_X_G473, G47_X_G473)  0.473012   

        R2Adj  
69   0.689594  
70   0.689507  
67   0.689437  
58   0.686647  
59   0.686629  
57   0.686611  
5    0.686457  
3       0.685  
4    0.684998  
2    0.684994  
..        ...  
117  0.570821  
124  0.540088  
122  0.540046  
123  0.540013  
125  0.535336  
47   0.520606  
49   0.520606  
46   0.520604  
52   0.518476  
121  0.471972  

[126 rows x 3 columns]

<br>

# $I$

>## $Igfcf$

In [20]:
def Igfcf_Cheat(n):
    #Cnd_vars = ["BS-BCI", "BS-CCI-BAL", "B_C_X_MIG_NRG", "C29_C30", "BS-CCI-BAL"]
    Cnd_vars = ["BS-BCI", "BS-ICI-BAL", "F_CC11_X_CC113", "BS-CCI-BAL", 
                "PROD", "MIG_CAG", "B_C", "B_C_X_MIG_NRG", 
                "C29_C30", "C", "IS-PEI", "F_CC112", "MIG_NRG_X_E", "D"]    
    # IS-PEI, F_CC112, MIG_NRG_X_E, D
    fake_ns = list(itertools.combinations(Cnd_vars, n))
    fake_df = pd.DataFrame(index=["variables", "R2", "R2Adj"])
    counter = 0
    
    for fake_n in fake_ns:
        
        df_in = pd.DataFrame(sorted(list(set(namq_10_gdp.ID))), columns=["ID"])
        
        ###########################################################################################
        
        # Chain linked volumes, percentage change compared to same period in previous year
        var = "CLV_PCH_SM"
        selec = namq_10_gdp[(namq_10_gdp.NA_ITEM=="P51G")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
        df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Chain linked volumes, percentage change on previous period
        var = "CLV_PCH_PRE"
        selec = namq_10_gdp_lag[(namq_10_gdp_lag.NA_ITEM=="P51G")&(namq_10_gdp_lag.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
        df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Business Climate Indicator
        var = "BS-BCI"
        if var in fake_n:
            dic_BCI = dict(zip(ei_bsci_m_r2_1.TIME.values, ei_bsci_m_r2_1.Value.values))
            df_in[var] = df_in.ID.str[:-2].map(dic_BCI)
        
        # Industrial Confidence Indicator
        var = "BS-ICI-BAL"
        if var in fake_n:
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Building permits index - New residential buildings
        var = "IS-PEI"
        if var in fake_n:
            selec = ei_isbu_q[(ei_isbu_q.INDIC==var)&(ei_isbu_q.UNIT=="I2015")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")

        # Residential buildings, except residences for communities
        var = "F_CC11_X_CC113"
        if var in fake_n:
            selec = sts_cobp_q[(sts_cobp_q.NACE_R2==var)&(sts_cobp_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Two- and more dwelling buildings
        var = "F_CC112"
        if var in fake_n:
            selec = sts_cobp_q[(sts_cobp_q.NACE_R2==var)&(sts_cobp_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Construction confidence indicator
        var = "BS-CCI-BAL"
        if var in fake_n:
            selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Volume index of production
        var = "PROD"
        if var in fake_n:
            selec = sts_copr_q[(sts_copr_q.INDIC_BT==var)&(sts_copr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Production in industry: MIG - energy (except Section E)
        var = "MIG_NRG_X_E"
        if var in fake_n:
            selec =  sts_inpr_q[(sts_inpr_q.NACE_R2==var)&(sts_inpr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Production in industry: MIG - capital goods
        var = "MIG_CAG"
        if var in fake_n:
            selec = sts_inpr_q[(sts_inpr_q.NACE_R2==var)&(sts_inpr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Production in industry: Mining and quarrying; manufacturing
        var = "B_C"
        if var in fake_n:
            selec =  sts_inpr_q[(sts_inpr_q.NACE_R2==var)&(sts_inpr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Production in industry: Mining and quarrying; manufacturing (except MIG energy)
        var = "B_C_X_MIG_NRG"
        if var in fake_n:
            selec = sts_inpr_q[(sts_inpr_q.NACE_R2==var)&(sts_inpr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")

        ## Production in industry: Manufacturing
        #var = "C"
        #selec = sts_inpr_q[(sts_inpr_q.NACE_R2==var)&(sts_inpr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", "C_prod"]
        #df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Production in industry: Electricity, gas, steam and air conditioning supply
        var = "D"
        if var in fake_n:
            selec = sts_inpr_q[(sts_inpr_q.NACE_R2==var)&(sts_inpr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Manufacture of motor vehicles, trailers, semi-trailers and of other transport equipment
        var = "C29_C30"
        if var in fake_n:
            selec = sts_intv_q[(sts_intv_q.NACE_R2==var)&(sts_intv_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        # Manufacturing
        var = "C"
        if var in fake_n:
            selec = sts_inpr_q[(sts_inpr_q.NACE_R2==var)&(sts_inpr_q.UNIT=="I15")][["ID", "Value"]]; selec.columns = ["ID", var]
            df_in = pd.merge(df_in, selec, on='ID', how="inner")
        
        ###########################################################################################
        
        # Dummies
        countries = set(namq_10_gdp.GEO); countries.remove('PT')
        for country in sorted(list(countries)):
            df_in[country] = (df_in.ID.str[-2:] == country).astype(int)
        
        Igfcf = df_in.dropna().copy().set_index("ID")
        Igfcf = Igfcf.loc[:, (Igfcf != 0).any(axis=0)].copy()
        del var, selec, df_in
        
        ###########################################################################################
        
        # Save
        Igfcf.to_csv("./Igfcf.csv")
        
        ###########################################################################################
        
        # Regression
        X = sm.add_constant(Igfcf.loc[:, Igfcf.columns != "CLV_PCH_SM"].copy())
        y = Igfcf.loc[:, "CLV_PCH_SM"].copy()
        results = sm.OLS(y, X).fit()
        
        fake_df[counter] = [fake_n, results.rsquared, results.rsquared_adj]
        counter += 1
        if counter % 100 == 0:
            print(counter/len(fake_ns)*100)
        
    return(fake_df.transpose())

In [21]:
Igfcf_Cheat(1).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables         R2      R2Adj
3       (BS-CCI-BAL,)   0.188767   0.179036
1       (BS-ICI-BAL,)   0.171882   0.162019
0           (BS-BCI,)   0.151485   0.140013
8          (C29_C30,)   0.152018    0.13925
6              (B_C,)  0.0714955  0.0589603
2   (F_CC11_X_CC113,)  0.0646107  0.0520638
7    (B_C_X_MIG_NRG,)  0.0642902  0.0513907
11         (F_CC112,)  0.0619613  0.0492483
4             (PROD,)  0.0611066   0.048769
5          (MIG_CAG,)  0.0597175  0.0472946
9                (C,)  0.0591509  0.0467206
13               (D,)  0.0583367  0.0457638
10          (IS-PEI,)  0.0564403  0.0437132
12     (MIG_NRG_X_E,)  0.0546024  0.0421326

In [22]:
Igfcf_Cheat(2).sort_values(["R2Adj", "R2"], ascending=[False, False])

variables         R2      R2Adj
40         (BS-CCI-BAL, C29_C30)   0.374198   0.364691
19         (BS-ICI-BAL, C29_C30)   0.357486   0.347725
7              (BS-BCI, C29_C30)    0.29738   0.285982
70      (B_C_X_MIG_NRG, C29_C30)   0.290454   0.279574
2           (BS-BCI, BS-CCI-BAL)   0.257981   0.247063
64                (B_C, C29_C30)   0.255881   0.244576
39   (BS-CCI-BAL, B_C_X_MIG_NRG)    0.23987   0.229011
38             (BS-CCI-BAL, B_C)   0.234626   0.223347
18   (BS-ICI-BAL, B_C_X_MIG_NRG)   0.229539   0.218658
25  (F_CC11_X_CC113, BS-CCI-BAL)    0.22303   0.211764
..                           ...        ...        ...
83              (C, MIG_NRG_X_E)  0.0598376  0.0469712
58                  (MIG_CAG, C)  0.0597573  0.0469008
81                   (C, IS-PEI)  0.0607125  0.0468927
61        (MIG_CAG, MIG_NRG_X_E)  0.0595442  0.0466739
59             (MIG_CAG, IS-PEI)  0.0598386  0.0460059
32      (F_CC11_X_CC113, IS-PEI)  0.0590159  0.0456306
51                (PROD, IS-PEI)  0.0581998  0.0443837
86         (IS-PEI, MIG_NRG_X_E)  0.0577043  0.0438398
85             (IS-PEI, F_CC112)  0.0559853  0.0424125
90              (MIG_NRG_X_E, D)  0.0547324  0.0417899

[91 rows x 3 columns]

In [23]:
Igfcf_Cheat(3).sort_values(["R2Adj", "R2"], ascending=[False, False])

27.472527472527474
54.94505494505495
82.41758241758241


variables         R2      R2Adj
223   (BS-CCI-BAL, B_C_X_MIG_NRG, C29_C30)   0.475686   0.467427
217             (BS-CCI-BAL, B_C, C29_C30)    0.47526   0.466994
117             (BS-ICI-BAL, B_C, C29_C30)   0.449889   0.441223
123   (BS-ICI-BAL, B_C_X_MIG_NRG, C29_C30)   0.449752   0.441083
27           (BS-BCI, BS-CCI-BAL, C29_C30)   0.442971   0.433821
57        (BS-BCI, B_C_X_MIG_NRG, C29_C30)   0.427307   0.417808
93       (BS-ICI-BAL, BS-CCI-BAL, C29_C30)   0.425201   0.416083
202            (BS-CCI-BAL, PROD, C29_C30)   0.385552   0.375806
233               (BS-CCI-BAL, C29_C30, D)    0.38527   0.375512
6            (BS-BCI, BS-ICI-BAL, C29_C30)    0.38421   0.374095
..                                     ...        ...        ...
194  (F_CC11_X_CC113, IS-PEI, MIG_NRG_X_E)  0.0598051  0.0454397
303         (MIG_CAG, IS-PEI, MIG_NRG_X_E)  0.0593678  0.0450259
361                   (IS-PEI, F_CC112, D)  0.0591423  0.0444948
275            (PROD, IS-PEI, MIG_NRG_X_E)  0.0583018  0.0437602
362               (IS-PEI, MIG_NRG_X_E, D)  0.0580469  0.0436772
354                   (C, IS-PEI, F_CC112)  0.0582059  0.0436271
302             (MIG_CAG, IS-PEI, F_CC112)  0.0574091  0.0428179
193      (F_CC11_X_CC113, IS-PEI, F_CC112)  0.0566426  0.0425767
274                (PROD, IS-PEI, F_CC112)  0.0563593  0.0417969
360         (IS-PEI, F_CC112, MIG_NRG_X_E)  0.0557543  0.0410824

[364 rows x 3 columns]

In [24]:
Igfcf_Cheat(4).sort_values(["R2Adj", "R2"], ascending=[False, False])

9.990009990009991
19.980019980019982
29.97002997002997
39.960039960039964
49.95004995004995
59.94005994005994
69.93006993006993
79.92007992007993
89.91008991008991
99.9000999000999


variables         R2  \
145           (BS-BCI, BS-CCI-BAL, B_C_X_MIG_NRG, C29_C30)    0.53311   
139                     (BS-BCI, BS-CCI-BAL, B_C, C29_C30)   0.532746   
365       (BS-ICI-BAL, BS-CCI-BAL, B_C_X_MIG_NRG, C29_C30)   0.515449   
359                 (BS-ICI-BAL, BS-CCI-BAL, B_C, C29_C30)    0.51429   
760                (BS-CCI-BAL, B_C_X_MIG_NRG, C29_C30, D)   0.483042   
745                          (BS-CCI-BAL, B_C, C29_C30, D)   0.481902   
686             (BS-CCI-BAL, PROD, B_C_X_MIG_NRG, C29_C30)   0.479845   
680                       (BS-CCI-BAL, PROD, B_C, C29_C30)   0.479777   
530   (F_CC11_X_CC113, BS-CCI-BAL, B_C_X_MIG_NRG, C29_C30)   0.477558   
524             (F_CC11_X_CC113, BS-CCI-BAL, B_C, C29_C30)   0.477539   
...                                                    ...        ...   
873                         (PROD, IS-PEI, MIG_NRG_X_E, D)  0.0584122   
996                      (C, IS-PEI, F_CC112, MIG_NRG_X_E)  0.0584919   
581                (F_CC11_X_CC113, PROD, IS-PEI, F_CC112)  0.0581035   
813                       (PROD, MIG_CAG, IS-PEI, F_CC112)  0.0582185   
609             (F_CC11_X_CC113, MIG_CAG, IS-PEI, F_CC112)  0.0580308   
928                          (MIG_CAG, IS-PEI, F_CC112, D)  0.0574798   
667         (F_CC11_X_CC113, IS-PEI, F_CC112, MIG_NRG_X_E)  0.0574858   
927                (MIG_CAG, IS-PEI, F_CC112, MIG_NRG_X_E)  0.0569974   
871                   (PROD, IS-PEI, F_CC112, MIG_NRG_X_E)  0.0565109   
1000                     (IS-PEI, F_CC112, MIG_NRG_X_E, D)  0.0560506   

          R2Adj  
145    0.525146  
139    0.524776  
365    0.507478  
359    0.506301  
760    0.474533  
745    0.473374  
686    0.471289  
680     0.47122  
530    0.468875  
524    0.468855  
...         ...  
873   0.0433549  
996   0.0433316  
581   0.0430758  
813    0.042925  
609   0.0429242  
928   0.0425429  
667   0.0423093  
927    0.041813  
871   0.0411135  
1000  0.0408425  

[1001 rows x 3 columns]